In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.11335559564144626,0.3235822218585125,0.1923953…
  RigidBodyDynamics.Joint… => [0.7897507450718799]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [9]:
cache = MechanismStateCache(mechanism, x);

In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),0.561097891173349 - 0.5479515529585492im - 0.6089532889744426jm - 0.11871876001013529km,Vec(0.7714047889454392,0.6736190471400404,0.7942391908342752))

In [11]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(3.1412680578225083,-0.410508338238356,0.7136593054261438))

In [12]:
mass(mechanism)

0.7257469411976307

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(2.549604790128007,0.22320744382194269,0.3240179108192406))

In [14]:
motion_subspace(joint1, x.q[joint1])

RigidBodyDynamics.MotionSubspaceBasis{6,Float64}(RigidBodyDynamics.CartesianFrame3D("before_1"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("after_1"),FixedSizeArrays.Mat{3,6,Float64}(
    1.0 0.0 0.0 0.0 0.0 0.0
    0.0 1.0 0.0 0.0 0.0 0.0
    0.0 0.0 1.0 0.0 0.0 0.0
)
,FixedSizeArrays.Mat{3,6,Float64}(
    0.0 0.0 0.0 1.0 0.0 0.0
    0.0 0.0 0.0 0.0 1.0 0.0
    0.0 0.0 0.0 0.0 0.0 1.0
)
)

In [15]:
relative_twist(cache, body2, body1)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.6032464414896894,-0.02823824647285003,0.3188898572185843),Vec(0.08032828751783616,-0.6175804558071083,-0.2066454646715904))

In [16]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.0,0.0,0.0),Vec(0.0,0.0,0.0))